In [ ]:
# pip install basemap


In [2]:
import os
import sys
import pandas as pd
import numpy as np
import obspy
from tqdm import tqdm

from obspy.clients.fdsn import Client
from obspy.clients.fdsn import Client as FDSNClient

from mpl_toolkits.basemap import Basemap


from pnwstore.mseed import WaveformClient
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from obspy import UTCDateTime

from tqdm import tqdm
from obspy.geodetics import locations2degrees, degrees2kilometers

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '../'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from plot_utils import *
from qc_utils import * 

## Morton Catalog

In [3]:
# Read Morton's catalog
events_morton = pd.read_csv('../data/ds01.csv')
# Convert the TSTRING to datetime
events_morton['datetime'] = pd.to_datetime(events_morton['TSTRING'], format='%Y%m%d%H%M%S', utc=True)
# Get the events in the Morton catalog 
t1 = pd.Timestamp('2011-1-1 00:00:00.000000+0000', tz='UTC')
t2 = pd.Timestamp('2015-12-31 23:59:59.999999+0000', tz='UTC')

events_morton= events_morton.loc[(events_morton['datetime'] > t1) & (events_morton['datetime'] < t2) ]

events_morton.head()

,CI YEAR,TSTRING,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,LAT,LON,...,dist to nearest stn,tt RMS,ERH,ERZ,STRIKE,DIP,RAKE,PLATE DESIGNATION,TEMPLATE EVENT?,datetime
0,1.0,2.011073e+13,2011.0,7.0,26.0,1.0,2.0,7.37,47.3217,-123.2708,...,27.4,0.19,0.8,1.2,NaN,NaN,NaN,Interface,Catalog,2011-07-26 01:02:07+00:00
1,1.0,2.011073e+13,2011.0,7.0,26.0,1.0,2.0,7.72,44.2888,-124.3340,...,163.8,0.06,13.1,3.2,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 01:02:07+00:00
2,1.0,2.011073e+13,2011.0,7.0,26.0,1.0,2.0,8.56,44.3017,-124.3180,...,131.1,0.50,35.4,22.2,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 01:02:08+00:00
3,1.0,2.011073e+13,2011.0,7.0,26.0,7.0,31.0,2.17,48.2635,-124.9298,...,44.4,0.77,3.5,6.4,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 07:31:02+00:00
4,1.0,2.011073e+13,2011.0,7.0,26.0,9.0,50.0,27.63,48.3032,-124.9157,...,46.1,0.94,4.0,6.9,NaN,NaN,NaN,Upper Plate,T,2011-07-26 09:50:27+00:00


In [4]:
events_morton

,CI YEAR,TSTRING,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,LAT,LON,...,dist to nearest stn,tt RMS,ERH,ERZ,STRIKE,DIP,RAKE,PLATE DESIGNATION,TEMPLATE EVENT?,datetime
0,1.0,2.011073e+13,2011.0,7.0,26.0,1.0,2.0,7.37,47.3217,-123.2708,...,27.4,0.19,0.8,1.2,NaN,NaN,NaN,Interface,Catalog,2011-07-26 01:02:07+00:00
1,1.0,2.011073e+13,2011.0,7.0,26.0,1.0,2.0,7.72,44.2888,-124.3340,...,163.8,0.06,13.1,3.2,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 01:02:07+00:00
2,1.0,2.011073e+13,2011.0,7.0,26.0,1.0,2.0,8.56,44.3017,-124.3180,...,131.1,0.50,35.4,22.2,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 01:02:08+00:00
3,1.0,2.011073e+13,2011.0,7.0,26.0,7.0,31.0,2.17,48.2635,-124.9298,...,44.4,0.77,3.5,6.4,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 07:31:02+00:00
4,1.0,2.011073e+13,2011.0,7.0,26.0,9.0,50.0,27.63,48.3032,-124.9157,...,46.1,0.94,4.0,6.9,NaN,NaN,NaN,Upper Plate,T,2011-07-26 09:50:27+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,4.0,2.015101e+13,2015.0,10.0,7.0,8.0,1.0,50.83,40.5895,-124.0455,...,5.4,0.14,0.9,0.8,NaN,NaN,NaN,Slab,NaN,2015-10-07 08:01:50+00:00
5278,4.0,2.015101e+13,2015.0,10.0,7.0,8.0,7.0,8.40,40.5380,-123.7217,...,14.9,0.09,3.3,13.5,NaN,NaN,NaN,Upper Plate,NaN,2015-10-07 08:07:08+00:00
5279,4.0,2.015101e+13,2015.0,10.0,7.0,11.0,31.0,26.69,40.5822,-124.0432,...,5.6,0.05,1.7,0.8,NaN,NaN,NaN,Slab,NaN,2015-10-07 11:31:26+00:00
5280,4.0,2.015101e+13,2015.0,10.0,7.0,18.0,11.0,9.52,40.2710,-124.3777,...,7.9,0.19,0.7,0.2,NaN,NaN,NaN,Upper Plate,NaN,2015-10-07 18:11:09+00:00


## ANSS Catalog

In [5]:
events_anss = pd.read_csv('../data/datasets_anss/anss_2011-15.csv')
events_anss['datetime'] = pd.to_datetime(events_anss['time'], format='%Y-%m-%dT%H:%M:%S.%fZ', utc=True)
events_anss= events_anss.loc[(events_anss['datetime'] > t1) & (events_anss['datetime'] < t2) ]
events_anss

,Unnamed: 0,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,datetime
0,0,2011-01-01T00:21:06.570Z,38.809834,-122.793663,1.746,0.88,md,14.0,73.0,0.001802,...,"6 km WSW of Cobb, California",earthquake,0.30,0.56,0.010,16.0,automatic,nc,nc,2011-01-01 00:21:06.570000+00:00
1,1,2011-01-01T02:04:41.480Z,38.789001,-122.747002,-0.844,0.28,md,5.0,122.0,0.009910,...,"4 km SSW of Cobb, California",earthquake,0.42,2.39,0.200,6.0,automatic,nc,nc,2011-01-01 02:04:41.480000+00:00
2,2,2011-01-01T02:53:49.640Z,38.817333,-122.821167,1.929,1.03,md,40.0,44.0,0.010810,...,"8 km W of Cobb, California",earthquake,0.14,0.19,0.060,13.0,reviewed,nc,nc,2011-01-01 02:53:49.640000+00:00
3,3,2011-01-01T03:11:04.720Z,38.841835,-122.829002,1.076,0.43,md,7.0,77.0,0.013510,...,"9 km WNW of Cobb, California",earthquake,0.43,0.88,0.150,7.0,automatic,nc,nc,2011-01-01 03:11:04.720000+00:00
4,4,2011-01-01T03:32:55.630Z,38.835833,-122.807000,0.947,0.16,md,9.0,112.0,0.012610,...,"7 km WNW of Cobb, California",earthquake,0.33,0.72,0.123,2.0,reviewed,nc,nc,2011-01-01 03:32:55.630000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131527,131527,2015-12-31T22:15:46.650Z,38.837502,-122.825333,1.450,0.18,md,6.0,180.0,0.008108,...,"9 km W of Cobb, California",earthquake,0.66,1.08,0.110,2.0,automatic,nc,nc,2015-12-31 22:15:46.650000+00:00
131528,131528,2015-12-31T22:18:13.120Z,41.856400,-119.599200,8.700,1.40,ml,6.0,210.1,0.175000,...,"45 km E of Fort Bidwell, California",earthquake,NaN,3.40,0.210,3.0,reviewed,nn,nn,2015-12-31 22:18:13.120000+00:00
131529,131529,2015-12-31T23:19:21.650Z,38.823334,-122.765663,1.680,0.54,md,7.0,99.0,0.008108,...,"3 km W of Cobb, California",earthquake,0.50,1.54,0.030,2.0,automatic,nc,nc,2015-12-31 23:19:21.650000+00:00
131530,131530,2015-12-31T23:22:20.730Z,38.841000,-122.878166,1.730,0.77,md,8.0,95.0,0.007207,...,"12 km ENE of Cloverdale, California",earthquake,0.58,1.02,0.180,3.0,automatic,nc,nc,2015-12-31 23:22:20.730000+00:00


## Our Catalog

In [6]:
# Load all the pick assignments 
year = 'or_shelf_trench'
mycatalog_picks = pd.read_csv(f'../data/datasets_{year}/all_pick_assignments_{year}.csv')
mycatalog = mycatalog_picks.drop_duplicates(subset=['idx'])
# Convert the time series in all_pick_assignments to datetime
mycatalog['datetime'] = pd.to_datetime(mycatalog['time'], utc = True)
mycatalog=mycatalog.loc[(mycatalog['datetime'] > t1) & (mycatalog['datetime'] < t2) ]
mycatalog

/tmp/ipykernel_2190526/2481397545.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mycatalog['datetime'] = pd.to_datetime(mycatalog['time'], utc = True)


,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
0,0,2,2011-11-09 03:35:00.298074+00:00,-147.822563,-146.146124,12.890625,9,43.170218,-127.317869,12.890625,2,1341358,-0.018578,J25A,P,1.320810e+09,2011-11-09 03:35:00.298074+00:00
9,9,3,2011-11-09 17:24:51.096176+00:00,-131.087933,-173.323761,16.015625,11,42.928762,-127.105763,16.015625,3,1341385,0.318173,J25A,P,1.320860e+09,2011-11-09 17:24:51.096176+00:00
20,20,4,2011-11-12 17:36:53.321241+00:00,-145.591279,-136.133311,2.734375,7,43.260733,-127.293081,2.734375,4,1312581,-0.271465,J25A,P,1.321119e+09,2011-11-12 17:36:53.321241+00:00
27,27,6,2011-11-21 04:49:11.940621+00:00,78.652760,-5.966732,0.390625,6,44.442031,-124.512004,0.390625,6,1327999,-0.796843,J25A,P,1.321851e+09,2011-11-21 04:49:11.940621+00:00
33,33,8,2011-11-28 00:27:13.522219+00:00,79.768402,-17.409948,0.390625,6,44.338944,-124.499748,0.390625,8,1313411,-0.812164,J25A,P,1.322440e+09,2011-11-28 00:27:13.522219+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,2749,2697,2015-08-06 12:39:34.744191+00:00,39.605290,-83.208438,16.015625,6,43.750090,-125.008266,16.015625,2697,2456232,0.518571,FC03D,S,1.438865e+09,2015-08-06 12:39:34.744191+00:00
2755,2755,2713,2015-08-10 16:55:14.717457+00:00,-182.407465,-130.411703,36.328125,6,43.304191,-127.748091,36.328125,2713,1159330,-0.151071,J10D,P,1.439226e+09,2015-08-10 16:55:14.717457+00:00
2761,2761,2732,2015-08-16 08:21:50.618422+00:00,71.958908,-10.257938,49.609375,7,44.404115,-124.596673,49.609375,2732,912035,-0.518026,FC03D,P,1.439713e+09,2015-08-16 08:21:50.618422+00:00
2768,2768,2737,2015-08-22 13:16:53.628155+00:00,67.496340,24.071709,49.609375,6,44.713443,-124.648192,49.609375,2737,1040980,-0.159884,FC03D,P,1.440249e+09,2015-08-22 13:16:53.628155+00:00


## Run this loop

In [7]:
matched_events_mycatalog2morton = []
matched_times_morton2mycatalog = []
matched_events_morton2mycatalog = []
unmatched_times_morton2mycatalog = []
unmatched_events_morton2mycatalog = []
unmatched_events_mycatalog2morton_and_anss = []
matched_events_anss2mycatalog=[]
matched_events_mycatalog2anss = []
unmatched_times_anss2mycatalog = []
unmatched_events_anss2mycatalog = []
matched_times_anss2mycatalog =[]

time_threshold = 5 # in seconds
dist_threshold =25

lat_morton2mycatalog = []
lon_morton2mycatalog = []

lat_anss2mycatalog =[]
lon_anss2mycatalog = []

# A set to keep track of matched indices in mycatalog
matched_indices_morton = set()
matched_indices_anss = set()
count_c = 0
count_d = 0
# Loop over events in Morton's catalog
for i in range(len(events_morton)):
    t11 = events_morton.iloc[i]['datetime']
    olat = events_morton.iloc[i]['LAT']
    olon = events_morton.iloc[i]['LON']
    
                                       
    condition = (mycatalog['datetime'] >= t11 - pd.Timedelta(seconds=time_threshold)) & \
                     (mycatalog['datetime'] <= t11 + pd.Timedelta(seconds=time_threshold)) & \
                     (degrees2kilometers(locations2degrees(olat, olon, mycatalog['latitude'], mycatalog['longitude'])) <= dist_threshold)                        
    matched_df = mycatalog.loc[condition]

    if len(matched_df) == 1:
        print('matched idx of Morton: ',i)
        count_c+=1
        matched_times_morton2mycatalog.append(t11)
        matched_events_morton2mycatalog.append(events_morton.iloc[i])
        lat_morton2mycatalog.append(events_morton.iloc[i]['LAT'])
        lon_morton2mycatalog.append(events_morton.iloc[i]['LON'])
        matched_events_mycatalog2morton.append(matched_df)
        matched_indices_morton.update(matched_df['idx'])
    elif len(matched_df) > 1:
        print('matched idx of Morton: ',i)

        count_d+=1
        diffs = abs(matched_df['datetime'] - t11)
        closest_index = diffs.idxmin()
        closest_event = matched_df.loc[[closest_index]]
        matched_times_morton2mycatalog.append(t11)
        matched_events_morton2mycatalog.append(events_morton.iloc[i])
        lat_morton2mycatalog.append(events_morton.iloc[i]['LAT'])
        lon_morton2mycatalog.append(events_morton.iloc[i]['LON'])
        matched_events_mycatalog2morton.append(closest_event)
        matched_indices_morton.update(closest_event['idx'])
    else:
        unmatched_times_morton2mycatalog.append(t11)
        unmatched_events_morton2mycatalog.append(events_morton.iloc[i])

# All events in mycatalog not matched with Morton's catalog are unmatched
unmatched_indices_morton = set(mycatalog.idx) - matched_indices_morton
# unmatched_events_mycatalog2morton = mycatalog.iloc[list(unmatched_indices_morton)]
count_a =0
count_b =0
# Loop over events in ANSS catalog
for i in range(len(events_anss)):
    t11 = events_anss.iloc[i]['datetime']
    olat = events_anss.iloc[i]['latitude']
    olon = events_anss.iloc[i]['longitude']
    
                                       
    condition = (mycatalog['datetime'] >= t11 - pd.Timedelta(seconds=time_threshold)) & \
                     (mycatalog['datetime'] <= t11 + pd.Timedelta(seconds=time_threshold)) & \
                     (degrees2kilometers(locations2degrees(olat, olon, mycatalog['latitude'], mycatalog['longitude'])) <= dist_threshold)                        
    matched_df = mycatalog.loc[condition]
    
    if len(matched_df) == 1:
        
        count_a+=1
        matched_times_anss2mycatalog.append(t11)
        matched_events_anss2mycatalog.append(events_anss.iloc[i])
        lat_anss2mycatalog.append(events_anss.iloc[i]['latitude'])
        lon_anss2mycatalog.append(events_anss.iloc[i]['longitude'])
        matched_events_mycatalog2anss.append(matched_df)
        matched_indices_anss.update(matched_df['idx'])
        
    elif len(matched_df) > 1:
        count_b+=1
        diffs = abs(matched_df['datetime'] - t11)
        closest_index = diffs.idxmin()
        closest_event = matched_df.loc[[closest_index]]
        matched_times_anss2mycatalog.append(t11)
        matched_events_anss2mycatalog.append(events_anss.iloc[i])
        lat_anss2mycatalog.append(events_anss.iloc[i]['latitude'])
        lon_anss2mycatalog.append(events_anss.iloc[i]['longitude'])
        matched_events_mycatalog2anss.append(closest_event)
        matched_indices_anss.update(closest_event['idx'])
        
    else:
        unmatched_times_anss2mycatalog.append(t11)
        unmatched_events_anss2mycatalog.append(events_anss.iloc[i])

# All events in mycatalog not matched with ANSS catalog are unmatched
unmatched_indices_anss = set(mycatalog.idx) - matched_indices_anss

unmatched_indices_morton_and_anss = unmatched_indices_morton.intersection(unmatched_indices_anss)
print(len(mycatalog),len(unmatched_indices_morton_and_anss))

# unmatched_events_mycatalog2morton_and_anss = mycatalog.iloc[list(unmatched_indices_morton_and_anss)]
unmatched_events_mycatalog2morton_and_anss = mycatalog.loc[mycatalog['event_idx'].isin(list(unmatched_indices_morton_and_anss))]


print(f"length of mycatalog:{len(mycatalog)}")
print(f"length of events_morton:{len(events_morton)}")
print(f"length of events_anss:{len(events_anss)}")
print(f"matched_indices_morton:{len(matched_indices_morton)}")
print(f"matched_indices_anss:{len(matched_indices_anss)}")
print(f"unmatched_indices_morton:{len(unmatched_indices_morton)}")
print(f"unmatched_indices_anss:{len(unmatched_indices_anss)}")
print(f"unmatched_indices_morton_and_anss (new events):{len(unmatched_indices_morton_and_anss)}")


matched idx of Morton:  108
matched idx of Morton:  111
matched idx of Morton:  112
matched idx of Morton:  129
matched idx of Morton:  155
matched idx of Morton:  174
matched idx of Morton:  185
matched idx of Morton:  189
matched idx of Morton:  193
matched idx of Morton:  194
matched idx of Morton:  208
matched idx of Morton:  212
matched idx of Morton:  233
matched idx of Morton:  236
matched idx of Morton:  239
matched idx of Morton:  240
matched idx of Morton:  249
matched idx of Morton:  255
matched idx of Morton:  266
matched idx of Morton:  271
matched idx of Morton:  272
matched idx of Morton:  275
matched idx of Morton:  290
matched idx of Morton:  307
matched idx of Morton:  316
matched idx of Morton:  318
matched idx of Morton:  319
matched idx of Morton:  610
matched idx of Morton:  725
matched idx of Morton:  736
matched idx of Morton:  754
matched idx of Morton:  770
matched idx of Morton:  1761
matched idx of Morton:  1789
matched idx of Morton:  1796
matched idx of Mo

In [8]:
# new concat code
# Concatenate and clean up dataframes
if len(matched_events_mycatalog2morton)>0:
    matched_events_mycatalog2morton = pd.concat(matched_events_mycatalog2morton).reset_index(drop=True)
    matched_times_mycatalog2morton = matched_events_mycatalog2morton['datetime']
    lat_mycatalog2morton = matched_events_mycatalog2morton['latitude']
    lon_mycatalog2morton = matched_events_mycatalog2morton['longitude']

if len(matched_events_morton2mycatalog)>0:   
    matched_events_morton2mycatalog = pd.DataFrame(matched_events_morton2mycatalog).reset_index(drop=True)
    
if len(unmatched_events_anss2mycatalog)>0:
    unmatched_events_anss2mycatalog = pd.DataFrame(unmatched_events_anss2mycatalog).reset_index(drop=True)
    
if len(unmatched_events_morton2mycatalog)>0:
    unmatched_events_morton2mycatalog = pd.DataFrame(unmatched_events_morton2mycatalog).reset_index(drop=True)

if len(unmatched_events_mycatalog2morton_and_anss) > 0:
    unmatched_events_mycatalog2morton_and_anss = unmatched_events_mycatalog2morton_and_anss.reset_index(drop=True)

if len(matched_events_mycatalog2anss)>0:
    matched_events_mycatalog2anss = pd.concat(matched_events_mycatalog2anss).reset_index(drop=True)
    matched_times_mycatalog2anss = matched_events_mycatalog2anss['datetime']
    lat_mycatalog2anss = matched_events_mycatalog2anss['latitude']
    lon_mycatalog2anss = matched_events_mycatalog2anss['longitude']

if len(matched_events_anss2mycatalog)>0:   
    matched_events_anss2mycatalog = pd.DataFrame(matched_events_anss2mycatalog).reset_index(drop=True)
    matched_times_anss2mycatalog = matched_events_anss2mycatalog['datetime']

print(f"length of mycatalog:{len(mycatalog)}")
print(f"length of events_morton:{len(events_morton)}")
print(f"length of events_anss:{len(events_anss)}")
print(f"matched_events_mycatalog2morton:{len(matched_events_mycatalog2morton)}")
print(f"matched_events_morton2mycatalog:{len(matched_events_morton2mycatalog)}")
print(f"unmatched_events_anss2mycatalog:{len(unmatched_events_anss2mycatalog)}")
print(f"unmatched_events_morton2mycatalog:{len(unmatched_events_morton2mycatalog)}")
print(f"unmatched_events_mycatalog2morton_and_anss (new events):{len(unmatched_events_mycatalog2morton_and_anss)}")
print(f"matched_events_mycatalog2anss:{len(matched_events_mycatalog2anss)}")
print(f"matched_events_anss2mycatalog:{len(matched_events_anss2mycatalog)}")

length of mycatalog:384
length of events_morton:5282
length of events_anss:131532
matched_events_mycatalog2morton:56
matched_events_morton2mycatalog:56
unmatched_events_anss2mycatalog:131489
unmatched_events_morton2mycatalog:5226
unmatched_events_mycatalog2morton_and_anss (new events):312
matched_events_mycatalog2anss:43
matched_events_anss2mycatalog:43


In [9]:
# Save these three catalogs to csv files
matched_events_mycatalog2morton.to_csv(f'../data/datasets_{year}/matched_events_with_morton_mycatalog.csv')
matched_events_anss2mycatalog.to_csv(f'../data/datasets_{year}/matched_events_with_mycatalog_anss.csv')
matched_events_morton2mycatalog.to_csv(f'../data/datasets_{year}/matched_events_with_mycatalog_morton.csv')
unmatched_events_mycatalog2morton_and_anss.to_csv(f'../data/datasets_{year}/new_events.csv')
matched_events_mycatalog2anss.to_csv(f'../data/datasets_{year}/matched_events_with_anss_mycatalog.csv')
unmatched_events_morton2mycatalog.to_csv(f'../data/datasets_{year}/missing_events_from_mycatalog_morton.csv')
unmatched_events_anss2mycatalog.to_csv(f'../data/datasets_{year}/missing_events_from_mycatalog_anss.csv')


## Plot the Origin Times

### Compare with Morton's Catalog 

In [ ]:
# Make lists for plotting the 1:1 line
x = pd.date_range(start='2011-1-1', end='2015-12-31', periods=len(matched_times_morton2mycatalog))
y = pd.date_range(start='2011-1-1', end='2015-12-31', periods=len(matched_times_morton2mycatalog))

# Plot the scatter plot
plt.figure()
plt.scatter(matched_times_mycatalog2morton,matched_times_morton2mycatalog)
plt.plot(x,y, 'k--')
plt.xlabel('Origin Time of mycatalog')
plt.ylabel('Origin Time of Morton')
plt.title('Scatter Plot: Origin Times of Matched Events')

### Compare with the ANSS Catalog 

In [ ]:
# Make lists for plotting the 1:1 line
matched_times_mycatalog2anss = pd.to_datetime(matched_times_mycatalog2anss)
matched_times_anss2mycatalog = pd.to_datetime(matched_times_anss2mycatalog)

x = pd.date_range(start='2014-1-1', end='2014-12-31', periods=len(matched_times_anss2mycatalog))
y = pd.date_range(start='2014-1-1', end='2014-12-31', periods=len(matched_times_anss2mycatalog))

# Plot the scatter plot
plt.figure()
plt.scatter(matched_times_mycatalog2anss,matched_times_anss2mycatalog)
plt.plot(x,y, 'k--')
plt.xlabel('Origin Time of mycatalog')
plt.ylabel('Origin Time of Morton')
plt.title('Scatter Plot: Origin Times of Matched Events')

## Plot the Latitudes

### Compare with Morton's Catalog 

In [ ]:
# Plot the scatter plot
plt.figure()
lat_min = min(lat_mycatalog2morton)
lat_max = max(lat_mycatalog2morton)

# Make lists for plotting the 1:1 line
x_lat = np.linspace(lat_min, lat_max, 10)
y_lat = np.linspace(lat_min, lat_max, 10)
plt.scatter(lat_mycatalog2morton,lat_morton2mycatalog, s=15)
plt.plot(x_lat,y_lat, 'k--')
plt.xlabel('Latitudes of Our Catalog ($^\circ$)')
plt.ylabel('Latitudes of Morton et al. (2023) ($^\circ$)')
plt.title('Latitudes of Our Catalog vs.Latitudes of Morton et al. (2023)')

### Compare with the ANSS Catalog 

In [ ]:
# Plot the scatter plot
plt.figure()
lat_min = min(lat_mycatalog2anss)
lat_max = max(lat_mycatalog2anss)

# Make lists for plotting the 1:1 line
x_lat = np.linspace(lat_min, lat_max, 10)
y_lat = np.linspace(lat_min, lat_max, 10)
plt.scatter(lat_mycatalog2anss,lat_anss2mycatalog, s=15)
plt.plot(x_lat,y_lat, 'k--')
plt.xlabel('Latitudes of Our Catalog ($^\circ$)')
plt.ylabel('Latitudes of ANSS Catalog ($^\circ$)')
plt.title('Latitudes of Our Catalog vs. ANSS Catalog')

## Plot the Longitudes

### Compare with Morton's Catalog 

In [ ]:
# Plot the scatter plot
plt.figure()
lon_min = min(lon_mycatalog2morton)
lon_max = max(lon_mycatalog2morton)

# Make lists for plotting the 1:1 line
x_lon = np.linspace(lon_min, lon_max, 10)
y_lon = np.linspace(lon_min, lon_max, 10)
plt.scatter(lon_mycatalog2morton,lon_morton2mycatalog)
plt.plot(x_lon,y_lon, 'k--')
plt.xlabel('Longitudes of Our Catalog ($^\circ$)')
plt.ylabel('Longitudes of Morton et al. (2023) ($^\circ$)')
plt.title('Longitudes of Our Catalog vs.Latitudes of Morton et al. (2023)')

### Compare with the ANSS Catalog 

In [ ]:
# Plot the scatter plot
plt.figure()
lon_min = min(lon_mycatalog2anss)
lon_max = max(lon_mycatalog2anss)

# Make lists for plotting the 1:1 line
x_lon = np.linspace(lon_min, lon_max, 10)
y_lon = np.linspace(lon_min, lon_max, 10)
plt.scatter(lon_mycatalog2anss,lon_anss2mycatalog)
plt.plot(x_lon,y_lon, 'k--')
plt.xlabel('Longitudes of Our Catalog ($^\circ$)')
plt.ylabel('Longitudes of Morton et al. (2023) ($^\circ$)')
plt.title('Longitudes of Our Catalog vs.Latitudes of Morton et al. (2023)')

## Histograms
Make histograms of mycatalog vs. Morton's catalog

### Histogram: All of our catalog 

In [ ]:
events = pd.read_csv('../../data/datasets_all_years/events_pnsn_wa.csv')

In [ ]:
# Plot the same histogram but overlay the histograms
bins = np.linspace(5,30,25)

plt.hist(unmatched_events_mycatalog2morton_and_anss['picks'], bins=bins,  
         alpha=0.5, # the transaparency parameter 
         label='Newly detected events') 
plt.hist(matched_events_mycatalog2anss['picks'], bins=bins,
         alpha=0.5, 
         label='Events matching the ANSS Catalog')   
plt.hist(matched_events_mycatalog2morton['picks'], bins=bins,
         alpha=0.5, 
         label='Events matching Morton et al., 2023') 

plt.xlabel('Number of Picks')
plt.ylabel('Number of Events')  
plt.legend(loc='upper right') 
plt.title('Picks vs. Number of Events') 
plt.show()


### Histogram: Matched events alone

### Compare with Morton et al. (2023)

In [ ]:
# Plot the histogram of the number of picks for the matched events
plt.figure()
bins = np.linspace(5,30,25)
plt.hist(matched_events_mycatalog2morton['picks'],bins=bins)
plt.xlabel('Number of Picks')
plt.ylabel('Number of Events')
plt.title('Histogram of Matched Events with Morton et al. (2023): Picks vs. the Number of Events')

### Compare with ANSS 

In [ ]:
# Plot the histogram of the number of picks for the matched events
plt.figure()
bins = np.linspace(5,30,25)
plt.hist(matched_events_mycatalog2anss['picks'],bins=bins)
plt.xlabel('Number of Picks')
plt.ylabel('Number of Events')
plt.title('Histogram of Matched Events with ANSS: Picks vs. the Number of Events')

## 2. Filter events 
Filter Events based on the following criteria
1. For an event, at least two stations have to be less than 50 km from the event and no station can be 100 km apart from each other.
2. An event has to have more than or equal to 6 picks


In [10]:
# Read the new events file
events = pd.read_csv(f'../data/datasets_{year}/new_events.csv')
events

,Unnamed: 0.1,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
0,0,0,2,2011-11-09 03:35:00.298074+00:00,-147.822563,-146.146124,12.890625,9,43.170218,-127.317869,12.890625,2,1341358,-0.018578,J25A,P,1.320810e+09,2011-11-09 03:35:00.298074+00:00
1,1,9,3,2011-11-09 17:24:51.096176+00:00,-131.087933,-173.323761,16.015625,11,42.928762,-127.105763,16.015625,3,1341385,0.318173,J25A,P,1.320860e+09,2011-11-09 17:24:51.096176+00:00
2,2,20,4,2011-11-12 17:36:53.321241+00:00,-145.591279,-136.133311,2.734375,7,43.260733,-127.293081,2.734375,4,1312581,-0.271465,J25A,P,1.321119e+09,2011-11-12 17:36:53.321241+00:00
3,3,47,10,2011-12-03 05:12:45.071214+00:00,-285.046527,-81.778036,0.390625,7,43.709274,-129.036594,0.390625,10,1317756,0.059037,J33A,P,1.322889e+09,2011-12-03 05:12:45.071214+00:00
4,4,54,13,2011-12-04 15:56:54.356666+00:00,-125.509723,-150.437330,0.390625,11,43.135636,-127.042608,0.390625,13,1338788,0.185487,J25A,P,1.323014e+09,2011-12-04 15:56:54.356666+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,307,2737,2683,2015-08-04 00:26:56.485913+00:00,234.842638,-224.818232,16.015625,6,42.440924,-122.645792,16.015625,2683,2504487,-0.877951,FC03D,S,1.438648e+09,2015-08-04 00:26:56.485913+00:00
308,308,2743,2689,2015-08-04 14:27:43.776389+00:00,71.958908,-44.587585,19.921875,6,44.095203,-124.601389,19.921875,2689,2516080,-0.530597,J10D,S,1.438698e+09,2015-08-04 14:27:43.776389+00:00
309,309,2749,2697,2015-08-06 12:39:34.744191+00:00,39.605290,-83.208438,16.015625,6,43.750090,-125.008266,16.015625,2697,2456232,0.518571,FC03D,S,1.438865e+09,2015-08-06 12:39:34.744191+00:00
310,310,2755,2713,2015-08-10 16:55:14.717457+00:00,-182.407465,-130.411703,36.328125,6,43.304191,-127.748091,36.328125,2713,1159330,-0.151071,J10D,P,1.439226e+09,2015-08-10 16:55:14.717457+00:00


In [11]:
all_pick_assignments = pd.read_csv(f'../data/datasets_{year}/all_pick_assignments_{year}.csv')
mycatalog = all_pick_assignments.drop_duplicates(subset=['idx'])
mycatalog

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick
0,0,2,2011-11-09 03:35:00.298074+00:00,-147.822563,-146.146124,12.890625,9,43.170218,-127.317869,12.890625,2,1341358,-0.018578,J25A,P,1.320810e+09
9,9,3,2011-11-09 17:24:51.096176+00:00,-131.087933,-173.323761,16.015625,11,42.928762,-127.105763,16.015625,3,1341385,0.318173,J25A,P,1.320860e+09
20,20,4,2011-11-12 17:36:53.321241+00:00,-145.591279,-136.133311,2.734375,7,43.260733,-127.293081,2.734375,4,1312581,-0.271465,J25A,P,1.321119e+09
27,27,6,2011-11-21 04:49:11.940621+00:00,78.652760,-5.966732,0.390625,6,44.442031,-124.512004,0.390625,6,1327999,-0.796843,J25A,P,1.321851e+09
33,33,8,2011-11-28 00:27:13.522219+00:00,79.768402,-17.409948,0.390625,6,44.338944,-124.499748,0.390625,8,1313411,-0.812164,J25A,P,1.322440e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,2749,2697,2015-08-06 12:39:34.744191+00:00,39.605290,-83.208438,16.015625,6,43.750090,-125.008266,16.015625,2697,2456232,0.518571,FC03D,S,1.438865e+09
2755,2755,2713,2015-08-10 16:55:14.717457+00:00,-182.407465,-130.411703,36.328125,6,43.304191,-127.748091,36.328125,2713,1159330,-0.151071,J10D,P,1.439226e+09
2761,2761,2732,2015-08-16 08:21:50.618422+00:00,71.958908,-10.257938,49.609375,7,44.404115,-124.596673,49.609375,2732,912035,-0.518026,FC03D,P,1.439713e+09
2768,2768,2737,2015-08-22 13:16:53.628155+00:00,67.496340,24.071709,49.609375,6,44.713443,-124.648192,49.609375,2737,1040980,-0.159884,FC03D,P,1.440249e+09


In [12]:
count = 0

In [13]:
"""
Inputs:
1. Either of the following files can be the input:
    1. matched_events_with_morton_mycatalog.csv from the 4_quality_control file
    2. matched_events_with_anss_mycatalog.csv from the 4_quality_control file
    3. new_events.csv from the 4_quality_control file
2. The all_pick_assignments CSV file from the 3_associate file: e.g., all_pick_assignments = pd.read_csv('../data/datasets_2012/all_pick_assignments_2012.csv')


Outputs:
1. A new dataframe that only has events that fall into the following categories:
    1. For an event, at least two stations have to be less than 50 km from the event and no station can be 100 km apart from each other.
2. An event has to have more than or equal to 6 picks
"""

# Parameters
client2 = Client("IRIS")

mycatalog = all_pick_assignments.drop_duplicates(subset=['idx'])
mycatalog['datetime'] = pd.to_datetime(mycatalog['time'], utc = True)

for i, idx in tqdm(enumerate(events['idx']),total=len(events['idx'])):
    event = mycatalog
    picks = all_pick_assignments
    picks_idx = picks.loc[picks['idx']==idx]
    pick_sta = np.unique(picks_idx['station'])
    otime = UTCDateTime(str(event[event['idx'] == idx]["datetime"].values[0]))
    distances = []
    max_dist = 10
    min_dist = 0
    for station in pick_sta:


        sta_inv = client2.get_stations(network='*',
                                       station=station, channel="?H?", 
                                       starttime=otime - 1e8, endtime=otime + 1e8,level="response")
        if len(sta_inv) == 0:
            print(f"Failed to fetch for {networks} {station} {otime}")
            continue

        _network = sta_inv[0].code
        slat = sta_inv[0][0].latitude
        slon = sta_inv[0][0].longitude
        olat = event.loc[event['idx']==idx, 'latitude'].values[0]
        olon = event.loc[event['idx']==idx, 'longitude'].values[0]

        dis1 = locations2degrees(olat, olon, slat, slon)
        dist = degrees2kilometers(dis1)

        distances.append([None, _network, station, dist])

    # Sort distances
    distances = sorted(distances, key=lambda item: item[-1])

    # This is for the first criterion in the markdown above
    # Determine if any two of the numbers in the distances list are less than or equal to 50
    found = False
    for i in range(len(distances)):
        for j in range(i + 1, len(distances)):
            if distances[i][3] <= 50 and distances[j][3] <= 50:
                found = True
                break
        if found:
            break

    # Make a list that includes the differences between the consecutive numbers in the distances list.
    differences = [distances[i+1][3] - distances[i][3] for i in range(len(distances) - 1)]

    if found == False: # If there were not at least two distances between the station and the event less than or equal to 50 km
        print(distances)
        index = events[events['idx'] == idx].index
        events = events.drop(index=index)

    elif any(differences > 100 for differences in differences): # If any of the distances between two stations were greater than 100 km
        print(distances)
        index = events[events['idx'] == idx].index
        events = events.drop(index=index)

    else: 
        continue


events 

/tmp/ipykernel_2190526/2718798803.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mycatalog['datetime'] = pd.to_datetime(mycatalog['time'], utc = True)
  0%|          | 1/312 [00:01<07:18,  1.41s/it]

[[None, '7D', 'M08A', 221.61978866820482], [None, '7D', 'J25A', 260.31146943298654], [None, '7D', 'M07A', 260.5562179258248], [None, '7D', 'J33A', 307.2374459432169], [None, '7D', 'J42A', 346.3188106861113], [None, '7D', 'J41A', 367.28224993641413]]


  1%|          | 2/312 [00:02<07:16,  1.41s/it]

[[None, '7D', 'M08A', 221.95476445205796], [None, '7D', 'J25A', 263.34218636383645], [None, '7D', 'M07A', 270.8406086818145], [None, '7D', 'J33A', 315.7695677118396], [None, '7D', 'J42A', 363.52142577623886], [None, '7D', 'J41A', 380.03161127970736]]


  1%|          | 3/312 [00:03<06:39,  1.29s/it]

...

[[None, '7D', 'J17D', 218.89798676762325], [None, '7D', 'J10D', 256.8214203696284], [None, '7D', 'J25D', 275.52173370061956], [None, '7D', 'FC03D', 312.94992576002073], [None, '7D', 'J26D', 335.3003880198216]]


100%|█████████▉| 311/312 [03:51<00:00,  3.11it/s]

[[None, '7D', 'J17D', 31.975555016378216], [None, '7D', 'J10D', 62.17578327618018], [None, '7D', 'J25D', 84.14781784686491], [None, '7D', 'J26D', 107.34138269138468], [None, '7D', 'FC03D', 120.16091589588326]]
[[None, '7D', 'J10D', 178.2621234376603], [None, '7D', 'J18D', 197.14612022902816], [None, '7D', 'J26D', 236.54869146389854]]


100%|██████████| 312/312 [03:52<00:00,  1.34it/s]


,Unnamed: 0.1,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
54,54,623,173,2012-08-13 05:12:53.305087+00:00,63.033772,-4.536330,12.890625,6,44.456431,-124.708007,12.890625,173,329032,0.137448,J25B,P,1.344835e+09,2012-08-13 05:12:53.305087+00:00
56,56,635,180,2012-08-17 07:50:47.584494+00:00,63.033772,-13.118742,16.015625,6,44.379203,-124.709049,16.015625,180,131576,1.405033,J25B,P,1.345190e+09,2012-08-17 07:50:47.584494+00:00
76,76,820,331,2012-12-05 17:22:37.351708+00:00,0.557821,-48.878791,49.609375,6,44.060117,-125.493038,49.609375,331,291359,0.367757,J10B,P,1.354728e+09,2012-12-05 17:22:37.351708+00:00
87,87,951,399,2014-01-30 01:34:09.590899+00:00,-19.523735,61.262160,16.015625,6,45.051010,-125.747836,16.015625,399,492330,-0.328593,J26C,P,1.391046e+09,2014-01-30 01:34:09.590899+00:00
103,103,1117,428,2014-03-01 22:36:00.566426+00:00,25.101945,157.099091,19.921875,6,45.913119,-125.176456,19.921875,428,613426,0.689580,J34C,P,1.393713e+09,2014-03-01 22:36:00.566426+00:00
110,110,1172,439,2014-03-12 00:32:25.317954+00:00,38.489648,129.921454,0.390625,6,45.667991,-125.006070,0.390625,439,1924521,0.666862,J26C,S,1.394584e+09,2014-03-12 00:32:25.317954+00:00
115,115,1206,451,2014-04-04 01:38:51.067878+00:00,75.305834,-17.409948,5.078125,6,44.339420,-124.555698,5.078125,451,1785154,0.253184,J25C,S,1.396576e+09,2014-04-04 01:38:51.067878+00:00
180,180,1839,841,2014-12-19 13:02:10.302626+00:00,-3.904747,-20.270752,49.609375,6,44.317568,-125.548946,49.609375,841,710290,-0.184959,J09D,P,1.418994e+09,2014-12-19 13:02:10.302626+00:00
207,207,2017,1244,2015-01-26 08:50:48.659080+00:00,80.884044,16.919699,1.953125,6,44.647709,-124.480392,1.953125,1244,2429425,-0.068733,FC03D,S,1.422262e+09,2015-01-26 08:50:48.659080+00:00
216,216,2071,1358,2015-02-01 11:37:01.611512+00:00,-13.945525,-86.069242,1.171875,6,43.725269,-125.673074,1.171875,1358,2216586,1.020591,FC03D,S,1.422791e+09,2015-02-01 11:37:01.611512+00:00


In [ ]:
new_events_filtered = filter_sta(new_events, mycatalog_picks)

In [14]:
events.to_csv(f'../data/datasets_{year}/new_events_filtered_{year}.csv')

In [15]:
events

,Unnamed: 0.1,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
54,54,623,173,2012-08-13 05:12:53.305087+00:00,63.033772,-4.536330,12.890625,6,44.456431,-124.708007,12.890625,173,329032,0.137448,J25B,P,1.344835e+09,2012-08-13 05:12:53.305087+00:00
56,56,635,180,2012-08-17 07:50:47.584494+00:00,63.033772,-13.118742,16.015625,6,44.379203,-124.709049,16.015625,180,131576,1.405033,J25B,P,1.345190e+09,2012-08-17 07:50:47.584494+00:00
76,76,820,331,2012-12-05 17:22:37.351708+00:00,0.557821,-48.878791,49.609375,6,44.060117,-125.493038,49.609375,331,291359,0.367757,J10B,P,1.354728e+09,2012-12-05 17:22:37.351708+00:00
87,87,951,399,2014-01-30 01:34:09.590899+00:00,-19.523735,61.262160,16.015625,6,45.051010,-125.747836,16.015625,399,492330,-0.328593,J26C,P,1.391046e+09,2014-01-30 01:34:09.590899+00:00
103,103,1117,428,2014-03-01 22:36:00.566426+00:00,25.101945,157.099091,19.921875,6,45.913119,-125.176456,19.921875,428,613426,0.689580,J34C,P,1.393713e+09,2014-03-01 22:36:00.566426+00:00
110,110,1172,439,2014-03-12 00:32:25.317954+00:00,38.489648,129.921454,0.390625,6,45.667991,-125.006070,0.390625,439,1924521,0.666862,J26C,S,1.394584e+09,2014-03-12 00:32:25.317954+00:00
115,115,1206,451,2014-04-04 01:38:51.067878+00:00,75.305834,-17.409948,5.078125,6,44.339420,-124.555698,5.078125,451,1785154,0.253184,J25C,S,1.396576e+09,2014-04-04 01:38:51.067878+00:00
180,180,1839,841,2014-12-19 13:02:10.302626+00:00,-3.904747,-20.270752,49.609375,6,44.317568,-125.548946,49.609375,841,710290,-0.184959,J09D,P,1.418994e+09,2014-12-19 13:02:10.302626+00:00
207,207,2017,1244,2015-01-26 08:50:48.659080+00:00,80.884044,16.919699,1.953125,6,44.647709,-124.480392,1.953125,1244,2429425,-0.068733,FC03D,S,1.422262e+09,2015-01-26 08:50:48.659080+00:00
216,216,2071,1358,2015-02-01 11:37:01.611512+00:00,-13.945525,-86.069242,1.171875,6,43.725269,-125.673074,1.171875,1358,2216586,1.020591,FC03D,S,1.422791e+09,2015-02-01 11:37:01.611512+00:00


In [ ]:
count = 0
for i, idx in enumerate(range(0,10,1)):
    print(i,idx,count)
    count+=1

print(count)

In [ ]:
new_events

## Plot using the plotting functions

In [ ]:
all_picks = pd.read_csv('../data/datasets_2014/all_picks_2014_for_assoc.csv')

In [ ]:
all_picks

In [ ]:
all_picks_networks = all_picks['station_network_code'].drop_duplicates()
list_networks = list(all_picks_networks)
all_picks_networks = ','.join(all_picks_networks)
all_picks_networks

In [ ]:
list_networks

In [ ]:
# Clean up the memory
del all_picks

## Plot the offshore events north of 44&deg; N

In [ ]:
new_events = pd.read_csv('../data/datasets_2014/new_events.csv')

In [ ]:
new_events_44N_and_above= new_events.loc[(new_events['latitude']>44)&(new_events['longitude']<-124)]
new_events_44N_and_above = new_events_44N_and_above[0:50] 

In [ ]:
# parameters for subplots_cluster_scale
mycatalog= mycatalog
mycatalog_picks=mycatalog_picks
networks= all_picks_networks
channel= "?H?"
idx_sta= 50
title= "Events matched"
fig_title= "new_events_filtered_44N_and_above_plots.pdf"

subplots_cluster_scale(new_events_44N_and_above,mycatalog_picks,networks,channel,idx_sta,title,fig_title)

## Plot one of the cluster events at around 46.5&deg; N, 125&deg; W using the plotting functions 

In [ ]:
def subplots_cluster_scale_p(idx, mycatalog, mycatalog_picks, networks, channel, idx_sta, title, fig_title):
    """
    idx: event_idx
    mycatalog: dataframe that contains only the unique picks (i.e., mycatalog_picks.drop_duplicates(subset=['idx']).copy())
    mycatalog_picks: all pick assignments csv file (e.g., pd.read_csv('../data/datasets_OR/all_pick_assignments_OR.csv'))
    networks: string of networks (e.g., "NV,OO,7A")
    channel: specify the direction of the channel (i.e., "?HZ", "?HE" or "?HN")
    idx_sta: choose the station to which you want to show the waveforms
    title: title in a string
    fig_title: figure title in as string
    """
        
    # Define the clients 
    client_waveform = WaveformClient()
    client2 = Client("IRIS")
    client_ncedc = Client('NCEDC')


    # Plot the earthquake moveout for one of the unmatched events for all stations 
    event = mycatalog
    picks = mycatalog_picks
    picks_idx = picks.loc[picks['idx']==idx]
    pick_sta = np.unique(picks_idx['station'])
    
    otime = UTCDateTime(str(event[event['idx'] == idx]["datetime"].values[0]))
    distances = []
    max_dist = 10
    min_dist = 0
    
    print(event[event['idx'] == idx]['picks'].values[0])
    for station in pick_sta:
        
        
        sta_inv = client2.get_stations(network=networks,
                                       station=station, channel="?H?", 
                                       starttime=otime - 1e8, endtime=otime + 1e8,level="response")
        if len(sta_inv) == 0:
#             print(f"Failed to fetch for {networks} {station} {otime}")
            continue
            
        _network = sta_inv[0].code
        slat = sta_inv[0][0].latitude
        slon = sta_inv[0][0].longitude
        olat = event.loc[event['idx']==idx, 'latitude'].values[0]
        olon = event.loc[event['idx']==idx, 'longitude'].values[0]
        
        dis1 = locations2degrees(olat, olon, slat, slon)
        dist = degrees2kilometers(dis1)
#         if max_dist < dist:
#             max_dist = dist
            
#         if min_dist > dist:
#             min_dist = dist
            
        distances.append([None, _network, station, dist])

    # Sort distances
    distances = sorted(distances, key=lambda item: item[-1])
    distances = distances[:idx_sta+1]
    
    # Set up to define the xlim and ylim
    max_y = 0
    min_y = 0
    # This count is for the if statements. Only used to ensure that min_y_count 
    #is changed from 0 to either the first positive value of the distance of one of the stations from the event
    min_y_count = 0 
    
    max_x = 0
    min_x = 0
    
    # This count is for the if statements. Only used to ensure that min_x_count 
    #is changed from 0 to either the first positive value of P pick time or the first positive value of S pick time
    min_x_count= 0
    # Create a figure
    fig,axs = plt.subplots(1,4,figsize=(18,6))
    gs = fig.add_gridspec(3, hspace=0, figure=fig)
#     axs = gs.subplots(sharex=True, sharey=True)
    starttime = otime -30
    endtime = otime + 120
    
    # Define texts
    texts = []
    
    for i, ii in enumerate(distances):
            
        if ii[1] in ['NC', 'BK']:
            # Query waveforms
            st = client_ncedc.get_waveforms(network=ii[1], station=ii[2], location="*", channel=channel,starttime=starttime, endtime=endtime)

        elif ii[1] in networks: 
            st = client_waveform.get_waveforms(network=ii[1], station=ii[2], channel=channel,starttime=starttime, endtime=endtime)
  
        else: 
            st =  Stream()
            print(f"WARNING: No data for {ii[1]}.{ii[2]}.{channel} on {otime}.")    
            continue
            
#         print(f"len(st):{len(st)}")
#         print(st)
    
        # Skip empty traces
        if len(st) == 0:
                continue
                
        sta_picks = picks_idx[picks_idx['station'] == ii[2]]
        p_picks = sta_picks.loc[sta_picks['phase'] == 'P']
        s_picks = sta_picks.loc[sta_picks['phase'] == 'S']
#         print(len(p_picks),len(s_picks))
        
        # Define the xlim values
        # Define the maximum x value
        if len(s_picks) > 0:
            if max_x < UTCDateTime(s_picks.iloc[0]['time_pick']) - starttime:
                max_x = UTCDateTime(s_picks.iloc[0]['time_pick']+5) - starttime
        elif len(p_picks) > 0:
            if max_x < UTCDateTime(p_picks.iloc[0]['time_pick']) - starttime: 
                max_x = UTCDateTime(p_picks.iloc[0]['time_pick']+5) - starttime
        else:
            print('No picks for this station. Skipping.')
            continue 
            
        # Define the minimum x value
        if len(p_picks) > 0:
            if min_x_count == 0:
                if min_x < UTCDateTime(p_picks.iloc[0]['time_pick']) - starttime:
                    min_x = UTCDateTime(p_picks.iloc[0]['time_pick']-5) - starttime
                    min_x_count += 1           
            else:
                if min_x >= UTCDateTime(p_picks.iloc[0]['time_pick']) - starttime:
                    min_x = UTCDateTime(p_picks.iloc[0]['time_pick']-5) - starttime            
        elif len(s_picks) > 0:
            if min_x_count == 0:
                if min_x < UTCDateTime(s_picks.iloc[0]['time_pick'])- starttime:
                    min_x = UTCDateTime(s_picks.iloc[0]['time_pick']-5)- starttime
                    min_x_count += 1                
            else:
                if min_x >= UTCDateTime(s_picks.iloc[0]['time_pick'])- starttime:
                    min_x = UTCDateTime(s_picks.iloc[0]['time_pick']-5) - starttime
        else:
            print('No picks for this station. Skipping.')
            continue    
            
        if len(p_picks) == 0:
            continue
            
#         print('This is after the p_pick continue statement')
    
        # Define ylim values
        if min_y_count == 0:
            if min_y < ii[3]:
                min_y = ii[3] - 5
                min_y_count += 1           
        else:
            if min_y >= ii[3]:
                min_y = ii[3] - 5 
                
        max_y = ii[3] + 5
        
    scaling_factor = (1/2)*(max_y-min_y)   
        
    for i, ii in enumerate(distances):
            
        if ii[1] in ['NC', 'BK']:
            # Query waveforms
            st = client_ncedc.get_waveforms(network=ii[1], station=ii[2], location="*", channel=channel,starttime=starttime, endtime=endtime)

        elif ii[1] in networks: 
            st = client_waveform.get_waveforms(network=ii[1], station=ii[2], channel=channel,starttime=starttime, endtime=endtime)
  
        else: 
            st =  Stream()
            print(f"WARNING: No data for {ii[1]}.{ii[2]}.{channel} on {otime}.")    
            continue
            
#         print(f"len(st):{len(st)}")
#         print(st)
    
        # Skip empty traces
        if len(st) == 0:
                continue
        _st = Stream()
        # Check for HH and BH channels presence
        has_HH = bool(st.select(channel="HH?"))
        has_BH = bool(st.select(channel="BH?"))

        # Apply selection logic based on channel presence
        if has_HH and has_BH:
            # If both HH and BH channels are present, select only HH
            _st += st.select(channel="HH?")
        elif has_HH:
            # If only HH channels are present
            _st += st.select(channel="HH?")
        elif has_BH:
            # If only BH channels are present
            _st += st.select(channel="BH?")

        st = _st

        print(f'Second st print:{_st}')
              
        st = Stream(filter(lambda st: st.stats.sampling_rate > 10, st))
        st.taper(max_percentage=0.05)
        st.filter(type='bandpass', freqmin=2, freqmax=25)
        st.merge(fill_value='interpolate') # fill gaps if there are any.

#         print(st)
        # Select only one trace per channel
        unique_channels = set(tr.stats.channel for tr in st)
        selected_traces = []
        
        for ch in unique_channels:
            selected_traces.append(next(tr for tr in st if tr.stats.channel == ch))
        st = Stream(selected_traces)
                   
        trim_st = st.copy()
        sta_picks = picks_idx[picks_idx['station'] == ii[2]]
        p_picks = sta_picks.loc[sta_picks['phase'] == 'P']
        s_picks = sta_picks.loc[sta_picks['phase'] == 'S']
#         print(len(p_picks),len(s_picks))
        
        
                
            
        if len(p_picks) == 0:
            continue 
#         print('This is after the p_pick continue statement')
        print(trim_st)
        
        for iax in range(len(trim_st)):
            print(iax)
            sampling_rate = trim_st[iax].stats.sampling_rate
            trim_st = trim_st.normalize()
            
            tp = UTCDateTime(p_picks.iloc[0]['time_pick']) - otime + 30
            i1 = int((tp-5)*sampling_rate)
            i2 = int((tp+15)*sampling_rate)

            offsets1 = ii[3]
            try: 
                wave = trim_st[iax].data
                wave = wave / (np.nanmax(wave[i1:i2], axis=-1)*10)
            except:
                continue 
            
#             print(trim_st[iax].stats.sampling_rate)
            axs[iax].plot(trim_st[iax].times(), wave * scaling_factor + offsets1, 
                          color='black', label=f"{trim_st[iax].stats.channel}", alpha=0.7, lw=0.5)
#             axs[iax].plot(trim_st[iax].times(), wave * 30 + offsets1, color='black', label=f"{trim_st[iax].stats.channel}", alpha=0.7, lw=0.5)

#             axs[iax].text(xlim[-1] + 2,   offsets1, 
#                               [ii[2]], fontsize=8, verticalalignment='bottom')

            if len(p_picks) > 0:
                axs[iax].vlines(UTCDateTime(p_picks.iloc[0]['time_pick']) - otime + 30, offsets1 - (1/35) * scaling_factor, 
                                offsets1 + (1/35) * scaling_factor, color='r')
            if len(s_picks) > 0:
                axs[iax].vlines(UTCDateTime(s_picks.iloc[0]['time_pick']) - otime + 30, offsets1 - (1/35) * scaling_factor, 
                                offsets1 + (1/35) * scaling_factor, color='b')
        texts.append([ii[2],ii[3]])

    
#     print(max_y,min_y)
    chs = ['2','1','Z']
    for iax in range(3):
        for i, ii in enumerate(texts):
            offsets1 = ii[1]
            axs[iax].text(max_x + 0.5, offsets1, 
                                  [ii[0]], fontsize=8, verticalalignment='bottom')
        axs[iax].legend(chs[iax],loc='upper right', handlelength=0)
        axs[iax].set_ylim([min_y,max_y])
        axs[iax].set_xlim([min_x,max_x])
        axs[iax].grid(alpha=0.5)
    fig.supxlabel('Time [sec]', y=0.07)
    fig.supylabel('Distance [km]')
    fig.suptitle(f"{title}: Origin Time={otime}, \n Latitude={round(event[event['idx']==idx]['latitude'].values[0], 2)}, Longtitude={round(event[event['idx']==idx]['longitude'].values[0], 2)}, Depth={round(event[event['idx']==idx]['depth'].values[0], 2)}", y=1)
    
    m = Basemap(projection='merc', llcrnrlat=40, urcrnrlat=50, llcrnrlon=-130, urcrnrlon=-120, resolution='i', ax=axs[3])
    m.drawcoastlines()
    m.drawcountries()
    m.drawstates()
    m.drawmapboundary()
    m.drawparallels(np.arange(40, 51, 1), labels=[1,0,0,0])
    m.drawmeridians(np.arange(-130, -119, 1), labels=[0,0,0,1])
    x, y = m(event[event['idx']==idx]['longitude'].values[0], event[event['idx']==idx]['latitude'].values[0])
    m.plot(x, y, 'ro', markersize=9)
    axs[3].set_title('Event Location')
    
    plt.show()


In [ ]:
def subplots_cluster_scale(idx, mycatalog, mycatalog_picks, networks, channel, idx_sta, title, fig_title):
    """
    idx: event_idx
    mycatalog: dataframe that contains only the unique picks (i.e., mycatalog_picks.drop_duplicates(subset=['idx']).copy())
    mycatalog_picks: all pick assignments csv file (e.g., pd.read_csv('../data/datasets_OR/all_pick_assignments_OR.csv'))
    networks: string of networks (e.g., "NV,OO,7A")
    channel: specify the direction of the channel (i.e., "?HZ", "?HE" or "?HN")
    idx_sta: choose the station to which you want to show the waveforms
    title: title in a string
    fig_title: figure title in as string
    """
        
    # Define the clients 
    client_waveform = WaveformClient()
    client2 = Client("IRIS")
    client_ncedc = Client('NCEDC')


    # Plot the earthquake moveout for one of the unmatched events for all stations 
    event = mycatalog
    picks = mycatalog_picks
    
    p = PdfPages(fig_title) 
    
    for idx in event['idx']:
        
        picks_idx = picks.loc[picks['idx']==idx]
        pick_sta = np.unique(picks_idx['station'])

        otime = UTCDateTime(str(event[event['idx'] == idx]["datetime"].values[0]))
        distances = []
        max_dist = 10
        min_dist = 0

        print(event[event['idx'] == idx]['picks'].values[0])
        for station in pick_sta:


            sta_inv = client2.get_stations(network=networks,
                                           station=station, channel="?H?", 
                                           starttime=otime - 1e8, endtime=otime + 1e8,level="response")
            if len(sta_inv) == 0:
    #             print(f"Failed to fetch for {networks} {station} {otime}")
                continue

            _network = sta_inv[0].code
            slat = sta_inv[0][0].latitude
            slon = sta_inv[0][0].longitude
            olat = event.loc[event['idx']==idx, 'latitude'].values[0]
            olon = event.loc[event['idx']==idx, 'longitude'].values[0]

            dis1 = locations2degrees(olat, olon, slat, slon)
            dist = degrees2kilometers(dis1)
    #         if max_dist < dist:
    #             max_dist = dist

    #         if min_dist > dist:
    #             min_dist = dist

            distances.append([None, _network, station, dist])

        # Sort distances
        distances = sorted(distances, key=lambda item: item[-1])
        distances = distances[:idx_sta+1]

        # Set up to define the xlim and ylim
        max_y = 0
        min_y = 0
        # This count is for the if statements. Only used to ensure that min_y_count 
        #is changed from 0 to either the first positive value of the distance of one of the stations from the event
        min_y_count = 0 

        max_x = 0
        min_x = 0

        # This count is for the if statements. Only used to ensure that min_x_count 
        #is changed from 0 to either the first positive value of P pick time or the first positive value of S pick time
        min_x_count= 0
        # Create a figure
        fig,axs = plt.subplots(1,4,figsize=(18,6))
        gs = fig.add_gridspec(3, hspace=0, figure=fig)
    #     axs = gs.subplots(sharex=True, sharey=True)
        starttime = otime -30
        endtime = otime + 120

        # Define texts
        texts = []

        for i, ii in enumerate(distances):

            if ii[1] in ['NC', 'BK']:
                # Query waveforms
                st = client_ncedc.get_waveforms(network=ii[1], station=ii[2], location="*", channel=channel,starttime=starttime, endtime=endtime)

            elif ii[1] in networks: 
                st = client_waveform.get_waveforms(network=ii[1], station=ii[2], channel=channel,starttime=starttime, endtime=endtime)

            else: 
                st =  Stream()
                print(f"WARNING: No data for {ii[1]}.{ii[2]}.{channel} on {otime}.")    
                continue

    #         print(f"len(st):{len(st)}")
    #         print(st)

            # Skip empty traces
            if len(st) == 0:
                    continue

            sta_picks = picks_idx[picks_idx['station'] == ii[2]]
            p_picks = sta_picks.loc[sta_picks['phase'] == 'P']
            s_picks = sta_picks.loc[sta_picks['phase'] == 'S']
    #         print(len(p_picks),len(s_picks))

            # Define the xlim values
            # Define the maximum x value
            if len(s_picks) > 0:
                if max_x < UTCDateTime(s_picks.iloc[0]['time_pick']) - starttime:
                    max_x = UTCDateTime(s_picks.iloc[0]['time_pick']+5) - starttime
            elif len(p_picks) > 0:
                if max_x < UTCDateTime(p_picks.iloc[0]['time_pick']) - starttime: 
                    max_x = UTCDateTime(p_picks.iloc[0]['time_pick']+5) - starttime
            else:
                print('No picks for this station. Skipping.')
                continue 

            # Define the minimum x value
            if len(p_picks) > 0:
                if min_x_count == 0:
                    if min_x < UTCDateTime(p_picks.iloc[0]['time_pick']) - starttime:
                        min_x = UTCDateTime(p_picks.iloc[0]['time_pick']-5) - starttime
                        min_x_count += 1           
                else:
                    if min_x >= UTCDateTime(p_picks.iloc[0]['time_pick']) - starttime:
                        min_x = UTCDateTime(p_picks.iloc[0]['time_pick']-5) - starttime            
            elif len(s_picks) > 0:
                if min_x_count == 0:
                    if min_x < UTCDateTime(s_picks.iloc[0]['time_pick'])- starttime:
                        min_x = UTCDateTime(s_picks.iloc[0]['time_pick']-5)- starttime
                        min_x_count += 1                
                else:
                    if min_x >= UTCDateTime(s_picks.iloc[0]['time_pick'])- starttime:
                        min_x = UTCDateTime(s_picks.iloc[0]['time_pick']-5) - starttime
            else:
                print('No picks for this station. Skipping.')
                continue    

    #         if len(p_picks) == 0:
    #             continue

    #         print('This is after the p_pick continue statement')

            # Define ylim values
            if min_y_count == 0:
                if min_y < ii[3]:
                    min_y = ii[3] - 5
                    min_y_count += 1           
            else:
                if min_y >= ii[3]:
                    min_y = ii[3] - 5 

            max_y = ii[3] + 5

        scaling_factor = (1/2)*(max_y-min_y)   

        for i, ii in enumerate(distances):

            if ii[1] in ['NC', 'BK']:
                # Query waveforms
                st = client_ncedc.get_waveforms(network=ii[1], station=ii[2], location="*", channel=channel,starttime=starttime, endtime=endtime)

            elif ii[1] in networks: 
                st = client_waveform.get_waveforms(network=ii[1], station=ii[2], channel=channel,starttime=starttime, endtime=endtime)

            else: 
                st =  Stream()
                print(f"WARNING: No data for {ii[1]}.{ii[2]}.{channel} on {otime}.")    
                continue

    #         print(f"len(st):{len(st)}")
    #         print(st)

            # Skip empty traces
            if len(st) == 0:
                    continue
            _st = Stream()
            # Check for HH and BH channels presence
            has_HH = bool(st.select(channel="HH?"))
            has_BH = bool(st.select(channel="BH?"))

            # Apply selection logic based on channel presence
            if has_HH and has_BH:
                # If both HH and BH channels are present, select only HH
                _st += st.select(channel="HH?")
            elif has_HH:
                # If only HH channels are present
                _st += st.select(channel="HH?")
            elif has_BH:
                # If only BH channels are present
                _st += st.select(channel="BH?")

            st = _st

            print(f'Second st print:{_st}')

            st = Stream(filter(lambda st: st.stats.sampling_rate > 10, st))
            st.taper(max_percentage=0.05)
            st.filter(type='bandpass', freqmin=2, freqmax=25)
            st.merge(fill_value='interpolate') # fill gaps if there are any.

    #         print(st)
            # Select only one trace per channel
            unique_channels = set(tr.stats.channel for tr in st)
            selected_traces = []

            for ch in unique_channels:
                selected_traces.append(next(tr for tr in st if tr.stats.channel == ch))
            st = Stream(selected_traces)

            trim_st = st.copy()
            sta_picks = picks_idx[picks_idx['station'] == ii[2]]
            p_picks = sta_picks.loc[sta_picks['phase'] == 'P']
            s_picks = sta_picks.loc[sta_picks['phase'] == 'S']
    #         print(len(p_picks),len(s_picks))




    #         if len(p_picks) == 0:
    #             continue 
    #         print('This is after the p_pick continue statement')
            print(trim_st)

            for iax in range(len(trim_st)):
                print(iax)
                sampling_rate = trim_st[iax].stats.sampling_rate
                trim_st = trim_st.normalize()

                if len(p_picks)>0:
                    tp = UTCDateTime(p_picks.iloc[0]['time_pick']) - otime + 30
                    i1 = int((tp-5)*sampling_rate)
                    i2 = int((tp+15)*sampling_rate)
                elif len(s_picks)>0:
                    ts = UTCDateTime(s_picks.iloc[0]['time_pick']) - otime + 30
                    i1 = int((ts-10)*sampling_rate)
                    i2 = int((ts+10)*sampling_rate)
                else:
                    print(f"WARNING: No pick time for {ii[1]}.{ii[2]}.{channel} on {otime}.")

                offsets1 = ii[3]
                try: 
                    wave = trim_st[iax].data
                    wave = wave / (np.nanmax(wave[i1:i2], axis=-1)*10)
                except:
                    continue 

    #             print(trim_st[iax].stats.sampling_rate)
                axs[iax].plot(trim_st[iax].times(), wave * scaling_factor + offsets1, 
                              color='black', label=f"{trim_st[iax].stats.channel}", alpha=0.7, lw=0.5)
    #             axs[iax].plot(trim_st[iax].times(), wave * 30 + offsets1, color='black', label=f"{trim_st[iax].stats.channel}", alpha=0.7, lw=0.5)

    #             axs[iax].text(xlim[-1] + 2,   offsets1, 
    #                               [ii[2]], fontsize=8, verticalalignment='bottom')

                if len(p_picks) > 0:
                    axs[iax].vlines(UTCDateTime(p_picks.iloc[0]['time_pick']) - otime + 30, offsets1 - (1/35) * scaling_factor, 
                                    offsets1 + (1/35) * scaling_factor, color='r')
                if len(s_picks) > 0:
                    axs[iax].vlines(UTCDateTime(s_picks.iloc[0]['time_pick']) - otime + 30, offsets1 - (1/35) * scaling_factor, 
                                    offsets1 + (1/35) * scaling_factor, color='b')
            texts.append([ii[2],ii[3]])


    #     print(max_y,min_y)
        chs = ['2','1','Z']
        for iax in range(3):
            for i, ii in enumerate(texts):
                offsets1 = ii[1]
                axs[iax].text(max_x + 0.5, offsets1, 
                                      [ii[0]], fontsize=8, verticalalignment='bottom')
            axs[iax].legend(chs[iax],loc='upper right', handlelength=0)
            axs[iax].set_ylim([min_y,max_y])
            axs[iax].set_xlim([min_x,max_x])
            axs[iax].grid(alpha=0.5)
        fig.supxlabel('Time [sec]', y=0.07)
        fig.supylabel('Distance [km]')
        fig.suptitle(f"{title}: Origin Time={otime}, \n Latitude={round(event[event['idx']==idx]['latitude'].values[0], 2)}, Longtitude={round(event[event['idx']==idx]['longitude'].values[0], 2)}, Depth={round(event[event['idx']==idx]['depth'].values[0], 2)}", y=1)

        m = Basemap(projection='merc', llcrnrlat=40, urcrnrlat=50, llcrnrlon=-130, urcrnrlon=-120, resolution='i', ax=axs[3])
        m.drawcoastlines()
        m.drawcountries()
        m.drawstates()
        m.drawmapboundary()
        m.drawparallels(np.arange(40, 51, 1), labels=[1,0,0,0])
        m.drawmeridians(np.arange(-130, -119, 1), labels=[0,0,0,1])
        x, y = m(event[event['idx']==idx]['longitude'].values[0], event[event['idx']==idx]['latitude'].values[0])
        m.plot(x, y, 'ro', markersize=9)
        axs[3].set_title('Event Location')
        
        fig.savefig(p, format='pdf')  

    p.close()

### Specify the latitudes and longitudes

In [ ]:
new_events = pd.read_csv('../data/datasets_2014/new_events.csv')

In [ ]:
idx = new_events[new_events['station']=='J25B']['idx'].values
idx

In [ ]:
new_events = new_events[new_events["idx"].isin(idx)]
new_events

In [ ]:
new_events_specific_lat_lon = new_events.loc[(new_events['latitude']>46)&(new_events['latitude']<47.5)&(new_events['longitude']<-124)&(new_events['longitude']>-126)]
new_events_specific_lat_lon

In [ ]:
# new_events = new_events_specific_lat_lon.loc[(new_events_specific_lat_lon['depth']>1)&(new_events_specific_lat_lon['depth']<40)]
new_events = new_events_specific_lat_lon.loc[new_events_specific_lat_lon['picks']>=6][0:30]
new_events

### Histogram: Depths of events

In [ ]:
# Plot the histogram of the number of picks for the matched events
plt.figure()
bins = np.linspace(0,50,25)
plt.hist(new_events['depth'],bins=bins)
plt.xlabel('Depth (km)')
plt.ylabel('Number of Events')
plt.title('Histogram of the depths of the events in the cluster at ~46.5$^\circ$ N')

### Plot HH?

In [ ]:
# parameters
mycatalog= mycatalog
mycatalog_picks=mycatalog_picks
networks= all_picks_networks
channel= "HH?"
idx_sta= 50
title= "Events matched"
fig_title= "events_matched.png"
ylim= [0,300]
xlim= [30,45]

for idx in new_events['idx']:
    subplots_cluster_scale_p(idx,mycatalog,mycatalog_picks,networks,channel,idx_sta,title,fig_title,ylim,xlim)

### Plot BH?

In [ ]:
# parameters
mycatalog= mycatalog
mycatalog_picks=mycatalog_picks
networks= all_picks_networks
channel= "BH?"
idx_sta= 50
title= "Events matched"
fig_title= "events_matched.png"
ylim= [0,300]
xlim= [30,45]

for idx in new_events['idx']:
    subplots_cluster_scale_p(idx,mycatalog,mycatalog_picks,networks,channel,idx_sta,title,fig_title,ylim,xlim)

### Plot both HH? and BH?

In [ ]:
new_events['datetime'] = pd.to_datetime(new_events['datetime'], utc = True)
new_events

In [ ]:
# parameters for subplots_cluster_scale_p
mycatalog= mycatalog
mycatalog_picks=mycatalog_picks
networks= all_picks_networks
channel= "?H?"
idx_sta= 50
title= "Events matched"
fig_title= "events_matched.png"

for idx in new_events['idx']:
    subplots_cluster_scale_p(idx,new_events,mycatalog_picks,networks,channel,idx_sta,title,fig_title)

In [ ]:
# parameters for subplots_cluster_scale
mycatalog= mycatalog
mycatalog_picks=mycatalog_picks
networks= all_picks_networks
channel= "?H?"
idx_sta= 50
title= "Events matched"
fig_title= "new_events_plots.pdf"

subplots_cluster_scale(new_events,mycatalog_picks,networks,channel,idx_sta,title,fig_title)

In [ ]:
# Plot the histogram of the number of picks for the matched events
new_events['time'] = pd.to_datetime(new_events['time'])
year1 = 2012
time1 = datetime(year=year1, month=1, day=1)
time2 = datetime(year=year1 + 1, month=1, day=1)
time_bins = pd.date_range(start=time1, end=time2, freq='5D')
plt.figure()
plt.hist(new_events['time'], bins=time_bins)
plt.xlabel('Days')
plt.ylabel('Number of Events')
plt.title('Histogram for the frequency of the events in the cluster at ~46.5$^\circ$ N')
plt.xticks(rotation=45)  # Optional: Rotate x-axis labels for better readability
plt.show()

In [ ]:
import matplotlib 
from matplotlib import pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages 

# customizing runtime configuration stored 
# in matplotlib.rcParams 
plt.rcParams["figure.figsize"] = [7.00, 3.50] 
plt.rcParams["figure.autolayout"] = True

fig1 = plt.figure() 
plt.plot([17, 45, 7, 8, 7], color='orange') 

fig2 = plt.figure() 
plt.plot([13, 25, 1, 6, 3], color='blue') 

Fig3 = plt.figure() 
plt.plot([22, 11, 2, 1, 23], color='green') 


def save_image(filename): 
	
	# PdfPages is a wrapper around pdf 
	# file so there is no clash and create 
	# files with no error. 
	p = PdfPages(filename) 
	
	# get_fignums Return list of existing 
	# figure numbers 
	fig_nums = plt.get_fignums() 
	figs = [plt.figure(n) for n in fig_nums] 
	
	# iterating over the numbers in list 
	for fig in figs: 
		
		# and saving the files 
		fig.savefig(p, format='pdf') 
	
	# close the object 
	p.close() 

# name your Pdf file 
filename = "multi_plot_image.pdf"

# call the function 
save_image(filename) 


## Plot the cluster events at around 43.25&deg; N, 124.25&deg; W using the plotting functions 

In [ ]:
new_events_specific_lat_lon = unmatched_events_mycatalog2morton_and_anss.loc[(unmatched_events_mycatalog2morton_and_anss['latitude']>43.0)&(unmatched_events_mycatalog2morton_and_anss['latitude']<43.3)&(unmatched_events_mycatalog2morton_and_anss['longitude']<-124.25)&(unmatched_events_mycatalog2morton_and_anss['longitude']>-124.75)]
new_events_specific_lat_lon

In [ ]:
new_events = new_events_specific_lat_lon.loc[new_events_specific_lat_lon['picks']>5]
new_events

### Plot HH?

In [ ]:
# parameters
mycatalog= mycatalog
mycatalog_picks=mycatalog_picks
networks= all_picks_networks
channel= "HH?"
idx_sta= 50
title= "Events matched"
fig_title= "events_matched.png"
ylim= [0,300]
xlim= [20,150]

for idx in new_events['idx']:
    subplots_cluster_scale_p(idx,mycatalog,mycatalog_picks,networks,channel,idx_sta,title,fig_title,ylim,xlim)

### Plot BH?

In [ ]:
# parameters
mycatalog= mycatalog
mycatalog_picks=mycatalog_picks
networks= all_picks_networks
channel= "BH?"
idx_sta= 50
title= "Events matched"
fig_title= "events_matched.png"
ylim= [0,300]
xlim= [20,150]

for idx in new_events['idx']:
    subplots_cluster_scale_p(idx,mycatalog,mycatalog_picks,networks,channel,idx_sta,title,fig_title,ylim,xlim)

## Calculate SNRs

In [ ]:
all_picks = pd.read_csv('../data/datasets_2014/all_picks_2014.csv')
all_pick_assignments = pd.read_csv('../data/datasets_2014/all_pick_assignments_2014.csv')

In [ ]:
def calc_snr(all_picks,all_pick_assignments):
    """ 
    This function calculates the SNRs for each station in each event.
    
    Inputs:
    1. The all_pick.csv file from the 2_format_pick2associate file.
    2. The all_pick_assignments.csv file from the 3_association file.
    
    Oututs:
    The new all_pick_assignments_snr file with the additional SNR column.
    """
    
    # Create the list of networks
    all_picks_networks = all_picks['station_network_code'].drop_duplicates()
    list_networks = list(all_picks_networks)
    networks = ','.join(all_picks_networks)
    
    
    
    # Define the clients 
    client_waveform = WaveformClient()
    client2 = Client("IRIS")
    client_ncedc = Client('NCEDC')
    
    all_pick_assignments['datetime'] = pd.to_datetime(all_pick_assignments['time'], utc = True)

    
    # Define parameters
    percentile=98
    
    # Create empty lists
    snr_list = []
        
    # Make sure if a station for an event has more than 1 P or S pick
    for idx in all_pick_assignments['idx'].drop_duplicates():
        # Define parameters
        otime = UTCDateTime(str(all_pick_assignments[all_pick_assignments['idx'] == idx]["datetime"].values[0]))


        # Create empty lists
        networks_stas = []

        # Plot the earthquake moveout for one of the unmatched events for all stations 
    #     event = mycatalog
        pick_idx = all_pick_assignments.loc[all_pick_assignments['idx']==idx]
        pick_sta = np.unique(pick_idx['station'])
        
    #     distances = []
    #     max_dist = 10
    #     min_dist = 0
        for station in pick_sta:

            sta_inv = client2.get_stations(network=networks,
                                           station=station, channel="?H?", 
                                           starttime=otime - 1e8, endtime=otime + 1e8,level="response")
            if len(sta_inv) == 0:
                print(f'No inventory for station {station}. Skipping.')
                continue

            network = sta_inv[0].code
    #         slat = sta_inv[0][0].latitude
    #         slon = sta_inv[0][0].longitude
    #         olat = event.loc[event['idx']==idx, 'latitude'].values[0]
    #         olon = event.loc[event['idx']==idx, 'longitude'].values[0]

    #         dis1 = locations2degrees(olat, olon, slat, slon)
    #         dist = degrees2kilometers(dis1)
    # #         if max_dist < dist:
    # #             max_dist = dist

    # #         if min_dist > dist:
    # #             min_dist = dist

            networks_stas.append([network,station])
        
   
        events = all_pick_assignments[all_pick_assignments['idx']==idx]
        for i in networks_stas:
            events1 = events[events['station']==i[1]]
            p_picks = events1[events1['phase']=='P']
            s_picks = events1[events1['phase']=='S']
            

            if len(p_picks)>0 and len(s_picks)>0:
                print(p_picks['datetime'].values,s_picks['datetime'].values)
                p_pick_time = UTCDateTime(str(p_picks['datetime'].values[0]))
                s_pick_time = UTCDateTime(str(s_picks['datetime'].values[0]))
                
                starttime_st = UTCDateTime(p_pick_time)-timedelta(seconds=120)
                endtime_st = UTCDateTime(p_pick_time)+timedelta(seconds=120)
                
                starttime_noise = UTCDateTime(p_pick_time)-timedelta(seconds=8)
                endtime_noise = UTCDateTime(p_pick_time)
                
                starttime_signal = UTCDateTime(s_pick_time)-timedelta(seconds=1)
                endtime_signal = UTCDateTime(s_pick_time)+timedelta(seconds=2)
                
                print(starttime_noise,endtime_noise)
            
                if i[0] in ['NC', 'BK']:
                # Query waveforms
                    st = client_ncedc.get_waveforms(network=i[0], station=i[1],
                                                    location="*", channel="?HZ",starttime=starttime_st,
                                                    endtime=endtime_st)

                elif i[0] in networks: 
                    st = client_waveform.get_waveforms(network=i[0], station=i[1],
                                                       channel='?HZ',starttime=starttime_st, endtime=endtime_st)

                else: 
                    st =  Stream()
                    print(f"WARNING: No data for {ii[1]}.{ii[2]}.{channel} on {otime}.")    
                    continue

        #         print(f"len(st):{len(st)}")
        #         print(st)

                # Skip empty traces
                if len(st) == 0:
                    continue
                    
                print(f'First st print:{st}')
                # Create a new stream
                _st = Stream()
                # Check for HH and BH channels presence
                has_HH = bool(st.select(channel="HH?"))
                has_BH = bool(st.select(channel="BH?"))

                # Apply selection logic based on channel presence
                if has_HH and has_BH:
                    # If both HH and BH channels are present, select only HH
                    _st += st.select(channel="HH?")
                elif has_HH:
                    # If only HH channels are present
                    _st += st.select(channel="HH?")
                elif has_BH:
                    # If only BH channels are present
                    _st += st.select(channel="BH?")

                _st.merge(fill_value='interpolate') # fill gaps if there are any.

                print(f'Second st print:{_st}')
            
                noise = _st.copy().trim(starttime=starttime_noise,endtime=endtime_noise)    
                signal = _st.copy().trim(starttime=starttime_signal,endtime=endtime_signal) 
                
                print(f"P and S pick times:{p_pick_time} and {s_pick_time}")
                print(f"Stream start and end times:{starttime_st} and {endtime_st}")
                print(f"Noise start and end times:{starttime_noise} and {endtime_noise}")
                print(f"Signal start and end times:{starttime_signal} and {endtime_signal}")
                
                print(f'Noise:{noise}')
                print(f'Signal:{signal}')

                noise_abs = np.percentile(abs(noise[0].data),percentile)
                signal_abs = np.percentile(abs(signal[0].data),percentile)

                snr = 20 * np.log10((signal_abs/noise_abs))

                snr_list.append(snr)
                
            if len(p_picks)>0 and len(s_picks)==0:
                
                print(p_picks['datetime'].values,s_picks['datetime'].values)
                p_pick_time = UTCDateTime(str(p_picks['datetime'].values[0]))
#                 s_pick_time = UTCDateTime(str(s_picks['datetime'].values[0]))

                starttime_st = UTCDateTime(p_pick_time)-timedelta(seconds=120)
                endtime_st = UTCDateTime(p_pick_time)+timedelta(seconds=120)
                
                starttime_noise = UTCDateTime(p_pick_time)-timedelta(seconds=8)
                endtime_noise = UTCDateTime(p_pick_time)
                
                starttime_signal = UTCDateTime(p_pick_time)+timedelta(seconds=1)
                endtime_signal = UTCDateTime(p_pick_time)+timedelta(seconds=4)
                
            
                if i[0] in ['NC', 'BK']:
                # Query waveforms
                    st = client_ncedc.get_waveforms(network=i[0], station=i[1],
                                                    location="*", channel="?HZ",starttime=starttime_st, 
                                                    endtime=endtime_st)

                elif i[0] in networks: 
                    st = client_waveform.get_waveforms(network=i[0], station=i[1],
                                                       channel='?HZ',starttime=starttime_st, endtime=endtime_st)

                else: 
                    st =  Stream()
                    print(f"WARNING: No data for {ii[1]}.{ii[2]}.{channel} on {otime}.")    
                    continue

        #         print(f"len(st):{len(st)}")
        #         print(st)

                # Skip empty traces
                if len(st) == 0:
                    continue
                print(f'First st print:{st}')
                # Create a new stream
                _st = Stream()
                # Check for HH and BH channels presence
                has_HH = bool(st.select(channel="HH?"))
                has_BH = bool(st.select(channel="BH?"))

                # Apply selection logic based on channel presence
                if has_HH and has_BH:
                    # If both HH and BH channels are present, select only HH
                    _st += st.select(channel="HH?")
                elif has_HH:
                    # If only HH channels are present
                    _st += st.select(channel="HH?")
                elif has_BH:
                    # If only BH channels are present
                    _st += st.select(channel="BH?")

                _st.merge(fill_value='interpolate') # fill gaps if there are any.

                print(f'Second st print:{_st}')
                
                
                    

                noise = _st.copy().trim(starttime=starttime_noise,endtime=endtime_noise)    
                signal = _st.copy().trim(starttime=starttime_signal,endtime=endtime_signal) 
                
                print(f"P pick time:{p_pick_time}")
                print(f"Stream start and end times:{starttime_st} and {endtime_st}")
                print(f"Noise start and end times:{starttime_noise} and {endtime_noise}")
                print(f"Signal start and end times:{starttime_signal} and {endtime_signal}")

                print(f'Noise:{noise}')
                print(f'Signal:{signal}')

                noise_abs = np.percentile(abs(noise[0].data),percentile)
                signal_abs = np.percentile(abs(signal[0].data),percentile)

                snr = 20 * np.log10((signal_abs/noise_abs))

                snr_list.append(snr)
                    
            if len(p_picks)==0 and len(s_picks)>0:
                print(p_picks['datetime'].values,s_picks['datetime'].values)
#                 p_pick_time = UTCDateTime(str(p_picks['datetime'].values))
                s_pick_time = UTCDateTime(str(s_picks['datetime'].values[0]))
    
                starttime_st = UTCDateTime(s_pick_time)-timedelta(seconds=120)
                endtime_st = UTCDateTime(s_pick_time)+timedelta(seconds=120)
                
                starttime_noise = UTCDateTime(s_pick_time)-timedelta(seconds=8)
                endtime_noise = UTCDateTime(s_pick_time)
                
                starttime_signal = UTCDateTime(s_pick_time)
                endtime_signal = UTCDateTime(s_pick_time)+timedelta(seconds=3)
                
                print(starttime_noise,endtime_noise)
            
                if i[0] in ['NC', 'BK']:
                # Query waveforms
                    st = client_ncedc.get_waveforms(network=i[0], station=i[1],
                                                    location="*", channel="?HZ",starttime=starttime_st, 
                                                    endtime=endtime_st)

                elif i[0] in networks: 
                    st = client_waveform.get_waveforms(network=i[0], station=i[1],
                                                       channel='?HZ',starttime=starttime_st, endtime=endtime_st)

                else: 
                    st =  Stream()
                    print(f"WARNING: No data for {ii[1]}.{ii[2]}.{channel} on {otime}.")    
                    continue

        #         print(f"len(st):{len(st)}")
        #         print(st)

                # Skip empty traces
                if len(st) == 0:
                    continue
                    
                print(f'First st print:{st}')
                # Create a new stream
                _st = Stream()
                # Check for HH and BH channels presence
                has_HH = bool(st.select(channel="HH?"))
                has_BH = bool(st.select(channel="BH?"))

                # Apply selection logic based on channel presence
                if has_HH and has_BH:
                    # If both HH and BH channels are present, select only HH
                    _st += st.select(channel="HH?")
                elif has_HH:
                    # If only HH channels are present
                    _st += st.select(channel="HH?")
                elif has_BH:
                    # If only BH channels are present
                    _st += st.select(channel="BH?")

                _st.merge(fill_value='interpolate') # fill gaps if there are any.

                print(f'Second st print:{_st}')

            
                noise = _st.copy().trim(starttime=starttime_noise,endtime=endtime_noise)    
                signal = _st.copy().trim(starttime=starttime_signal,endtime=endtime_signal) 
                
                print(f"S pick time:{s_pick_time}")
                print(f"Stream start and end times:{starttime_st} and {endtime_st}")
                print(f"Noise start and end times:{starttime_noise} and {endtime_noise}")
                print(f"Signal start and end times:{starttime_signal} and {endtime_signal}")
                
                print(f'Noise:{noise}')
                print(f'Signal:{signal}')

                noise_abs = np.percentile(abs(noise[0].data),percentile)
                signal_abs = np.percentile(abs(signal[0].data),percentile)

                snr = 20 * np.log10((signal_abs/noise_abs))

                snr_list.append(snr)
            
    all_assignments_picks['snr']=snr_list
     
    return snr_list

In [ ]:
snr_list = calc_snr(all_picks,all_pick_assignments)

In [ ]:
snr_list

In [ ]:
client_waveform = WaveformClient()

p_pick_time = '2014-01-02T07:40:02.642533Z'

starttime_st = UTCDateTime(p_pick_time)-timedelta(seconds=120)
endtime_st = UTCDateTime(p_pick_time)+timedelta(seconds=120)

# starttime_st = UTCDateTime('2011-01-03T00:00:0.000000Z')
# endtime_st = UTCDateTime('2011-01-03T12:59:59.000000Z')

print(f'Requested start and end times:{starttime_st} and {endtime_st}')


st = client_waveform.get_waveforms(network='CN', station='BTB',
                                       channel='HHZ',starttime=starttime_st, endtime=endtime_st)
print(f'Actual start and end times: {st[0].stats.starttime} and {st[0].stats.endtime}')
